<a href="https://colab.research.google.com/github/sndpkirwai/classification-projets/blob/main/Traffic_accident_prediction/traffic-accident-classification-updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
%matplotlib inline

In [99]:
rta_data = pd.read_csv("/content/drive/MyDrive/DS/RTA_classification/RTA Dataset.csv")
rta_data.head()

,Time,Day_of_week,Age_band_of_driver,Sex_of_driver,Educational_level,Vehicle_driver_relation,Driving_experience,Type_of_vehicle,Owner_of_vehicle,Service_year_of_vehicle,...,Vehicle_movement,Casualty_class,Sex_of_casualty,Age_band_of_casualty,Casualty_severity,Work_of_casuality,Fitness_of_casuality,Pedestrian_movement,Cause_of_accident,Accident_severity
0,17:02:00,Monday,18-30,Male,Above high school,Employee,1-2yr,Automobile,Owner,Above 10yr,...,Going straight,na,na,na,na,NaN,NaN,Not a Pedestrian,Moving Backward,Slight Injury
1,17:02:00,Monday,31-50,Male,Junior high school,Employee,Above 10yr,Public (> 45 seats),Owner,5-10yrs,...,Going straight,na,na,na,na,NaN,NaN,Not a Pedestrian,Overtaking,Slight Injury
2,17:02:00,Monday,18-30,Male,Junior high school,Employee,1-2yr,Lorry (41?100Q),Owner,NaN,...,Going straight,Driver or rider,Male,31-50,3,Driver,NaN,Not a Pedestrian,Changing lane to the left,Serious Injury
3,1:06:00,Sunday,18-30,Male,Junior high school,Employee,5-10yr,Public (> 45 seats),Governmental,NaN,...,Going straight,Pedestrian,Female,18-30,3,Driver,Normal,Not a Pedestrian,Changing lane to the right,Slight Injury
4,1:06:00,Sunday,18-30,Male,Junior high school,Employee,2-5yr,NaN,Owner,5-10yrs,...,Going straight,na,na,na,na,NaN,NaN,Not a Pedestrian,Overtaking,Slight Injury


In [100]:
%%capture
!pip install wandb

## Step 1: Import W&B and Login

In [101]:
import wandb

wandb.login()

True

In [102]:
rta_data.info

<bound method DataFrame.info of            Time Day_of_week Age_band_of_driver Sex_of_driver  \
0      17:02:00      Monday              18-30          Male   
1      17:02:00      Monday              31-50          Male   
2      17:02:00      Monday              18-30          Male   
3       1:06:00      Sunday              18-30          Male   
4       1:06:00      Sunday              18-30          Male   
...         ...         ...                ...           ...   
12311  16:15:00   Wednesday              31-50          Male   
12312  18:00:00      Sunday            Unknown          Male   
12313  13:55:00      Sunday            Over 51          Male   
12314  13:55:00      Sunday              18-30        Female   
12315  13:55:00      Sunday              18-30          Male   

        Educational_level Vehicle_driver_relation Driving_experience  \
0       Above high school                Employee              1-2yr   
1      Junior high school                Employee      

In [103]:
rta_data.columns

Index(['Time', 'Day_of_week', 'Age_band_of_driver', 'Sex_of_driver',
       'Educational_level', 'Vehicle_driver_relation', 'Driving_experience',
       'Type_of_vehicle', 'Owner_of_vehicle', 'Service_year_of_vehicle',
       'Defect_of_vehicle', 'Area_accident_occured', 'Lanes_or_Medians',
       'Road_allignment', 'Types_of_Junction', 'Road_surface_type',
       'Road_surface_conditions', 'Light_conditions', 'Weather_conditions',
       'Type_of_collision', 'Number_of_vehicles_involved',
       'Number_of_casualties', 'Vehicle_movement', 'Casualty_class',
       'Sex_of_casualty', 'Age_band_of_casualty', 'Casualty_severity',
       'Work_of_casuality', 'Fitness_of_casuality', 'Pedestrian_movement',
       'Cause_of_accident', 'Accident_severity'],
      dtype='object')

In [104]:
rta_data.isna().sum()

Time                              0
Day_of_week                       0
Age_band_of_driver                0
Sex_of_driver                     0
Educational_level               741
Vehicle_driver_relation         579
Driving_experience              829
Type_of_vehicle                 950
Owner_of_vehicle                482
Service_year_of_vehicle        3928
Defect_of_vehicle              4427
Area_accident_occured           239
Lanes_or_Medians                385
Road_allignment                 142
Types_of_Junction               887
Road_surface_type               172
Road_surface_conditions           0
Light_conditions                  0
Weather_conditions                0
Type_of_collision               155
Number_of_vehicles_involved       0
Number_of_casualties              0
Vehicle_movement                308
Casualty_class                    0
Sex_of_casualty                   0
Age_band_of_casualty              0
Casualty_severity                 0
Work_of_casuality           

In [105]:
rta_data_corr = rta_data.corr()
rta_data_corr.style.background_gradient(cmap="cool")

,Number_of_vehicles_involved,Number_of_casualties
Number_of_vehicles_involved,1.000000,0.213427
Number_of_casualties,0.213427,1.000000


In [106]:
# separate into input and output columns
features = rta_data.iloc[:,:-1]
print(features)
outputLabel = rta_data.iloc[:,-1]
print(outputLabel)
# pip install sklearn-pandas==1.5.0

           Time Day_of_week Age_band_of_driver Sex_of_driver  \
0      17:02:00      Monday              18-30          Male   
1      17:02:00      Monday              31-50          Male   
2      17:02:00      Monday              18-30          Male   
3       1:06:00      Sunday              18-30          Male   
4       1:06:00      Sunday              18-30          Male   
...         ...         ...                ...           ...   
12311  16:15:00   Wednesday              31-50          Male   
12312  18:00:00      Sunday            Unknown          Male   
12313  13:55:00      Sunday            Over 51          Male   
12314  13:55:00      Sunday              18-30        Female   
12315  13:55:00      Sunday              18-30          Male   

        Educational_level Vehicle_driver_relation Driving_experience  \
0       Above high school                Employee              1-2yr   
1      Junior high school                Employee         Above 10yr   
2      Junior h

In [107]:
# As the columns which have missing values, they are only categorical, we'll use the categorical imputer
# Importing the categorical imputer
# from sklearn_pandas import CategoricalImputer
# imputer = CategoricalImputer()
imputer = SimpleImputer(strategy='most_frequent')
imputed_features = pd.DataFrame(imputer.fit_transform(features))
imputed_features.columns = features.columns
imputed_features.columns

Index(['Time', 'Day_of_week', 'Age_band_of_driver', 'Sex_of_driver',
       'Educational_level', 'Vehicle_driver_relation', 'Driving_experience',
       'Type_of_vehicle', 'Owner_of_vehicle', 'Service_year_of_vehicle',
       'Defect_of_vehicle', 'Area_accident_occured', 'Lanes_or_Medians',
       'Road_allignment', 'Types_of_Junction', 'Road_surface_type',
       'Road_surface_conditions', 'Light_conditions', 'Weather_conditions',
       'Type_of_collision', 'Number_of_vehicles_involved',
       'Number_of_casualties', 'Vehicle_movement', 'Casualty_class',
       'Sex_of_casualty', 'Age_band_of_casualty', 'Casualty_severity',
       'Work_of_casuality', 'Fitness_of_casuality', 'Pedestrian_movement',
       'Cause_of_accident'],
      dtype='object')

In [108]:
imputed_features.isna().sum()

Time                           0
Day_of_week                    0
Age_band_of_driver             0
Sex_of_driver                  0
Educational_level              0
Vehicle_driver_relation        0
Driving_experience             0
Type_of_vehicle                0
Owner_of_vehicle               0
Service_year_of_vehicle        0
Defect_of_vehicle              0
Area_accident_occured          0
Lanes_or_Medians               0
Road_allignment                0
Types_of_Junction              0
Road_surface_type              0
Road_surface_conditions        0
Light_conditions               0
Weather_conditions             0
Type_of_collision              0
Number_of_vehicles_involved    0
Number_of_casualties           0
Vehicle_movement               0
Casualty_class                 0
Sex_of_casualty                0
Age_band_of_casualty           0
Casualty_severity              0
Work_of_casuality              0
Fitness_of_casuality           0
Pedestrian_movement            0
Cause_of_a

In [109]:
imputed_features['Type_of_vehicle'].value_counts(), rta_data['Day_of_week'].value_counts(), imputed_features['Age_band_of_driver'].value_counts(), imputed_features['Sex_of_driver'].value_counts(), imputed_features['Educational_level'].value_counts(),imputed_features['Vehicle_driver_relation'].value_counts(),imputed_features['Driving_experience'].value_counts(),imputed_features['Owner_of_vehicle'].value_counts(), imputed_features['Service_year_of_vehicle'].value_counts(), imputed_features['Defect_of_vehicle'].value_counts(), imputed_features['Area_accident_occured'].value_counts(),imputed_features['Lanes_or_Medians'].value_counts(), imputed_features['Road_allignment'].value_counts(), imputed_features['Types_of_Junction'].value_counts(),imputed_features['Fitness_of_casuality'].value_counts(), imputed_features['Work_of_casuality'].value_counts(), imputed_features['Vehicle_movement'].value_counts(), imputed_features['Type_of_collision'].value_counts(), imputed_features['Weather_conditions'].value_counts(), imputed_features['Light_conditions'].value_counts(), imputed_features['Road_surface_conditions'].value_counts(), imputed_features['Road_surface_type'].value_counts(), imputed_features['Casualty_class'].value_counts(), imputed_features['Sex_of_casualty'].value_counts(), imputed_features['Age_band_of_casualty'].value_counts(), imputed_features['Casualty_severity'].value_counts(), imputed_features['Pedestrian_movement'].value_counts(), imputed_features['Cause_of_accident'].value_counts()

(Automobile              4155
 Lorry (41?100Q)         2186
 Other                   1208
 Pick up upto 10Q         811
 Public (12 seats)        711
 Stationwagen             687
 Lorry (11?40Q)           541
 Public (13?45 seats)     532
 Public (> 45 seats)      404
 Long lorry               383
 Taxi                     265
 Motorcycle               177
 Special vehicle           84
 Ridden horse              76
 Turbo                     46
 Bajaj                     29
 Bicycle                   21
 Name: Type_of_vehicle, dtype: int64, Friday       2041
 Thursday     1851
 Wednesday    1840
 Tuesday      1770
 Monday       1681
 Saturday     1666
 Sunday       1467
 Name: Day_of_week, dtype: int64, 18-30       4271
 31-50       4087
 Over 51     1585
 Unknown     1548
 Under 18     825
 Name: Age_band_of_driver, dtype: int64, Male       11437
 Female       701
 Unknown      178
 Name: Sex_of_driver, dtype: int64, Junior high school    8360
 Elementary school     2163
 High school

In [110]:
imputed_features['Type_of_vehicle'].replace({np.nan: 'unknown'}, inplace=True)
imputed_features['Area_accident_occured'].replace({np.nan: 'unknown'}, inplace=True)
imputed_features['Road_surface_type'].replace({'na': 0}, inplace=True)
imputed_features['Casualty_class'].replace({'na': 0}, inplace=True)
imputed_features['Casualty_class'].replace({np.nan: 'unknown'}, inplace=True)
imputed_features['Casualty_class'].replace({0: 'unknown'}, inplace=True)

imputed_features['Sex_of_casualty'].replace({'na': 0}, inplace=True)
# imputed_features['Sex_of_casualty'].replace({np.nan: 'unknown'}, inplace=True)
imputed_features['Sex_of_casualty'].replace({0: 'unknown'}, inplace=True)

imputed_features['Age_band_of_casualty'].replace({'na': 0}, inplace=True)
imputed_features['Age_band_of_casualty'].replace({np.nan: 'unknown'}, inplace=True)
imputed_features['Age_band_of_casualty'].replace({0: 'unknown'}, inplace=True)

imputed_features['Casualty_severity'].replace({'na': 0}, inplace=True)
imputed_features['Casualty_severity'].replace({np.nan: 'unknown'}, inplace=True)
imputed_features['Casualty_severity'].replace({0: 'unknown'}, inplace=True)

imputed_features['Age_band_of_casualty'].replace({0: 'unknown'}, inplace=True)

imputed_features['Pedestrian_movement'].replace({np.nan: 'unknown'}, inplace=True)
imputed_features['Cause_of_accident'].replace({np.nan: 'unknown'}, inplace=True)

In [111]:
cat_col_list = ['Type_of_vehicle','Day_of_week','Age_band_of_driver','Sex_of_driver','Educational_level','Vehicle_driver_relation', 'Driving_experience','Owner_of_vehicle','Service_year_of_vehicle','Defect_of_vehicle','Area_accident_occured','Lanes_or_Medians','Road_allignment','Types_of_Junction','Fitness_of_casuality','Work_of_casuality','Vehicle_movement', 'Type_of_collision', 'Weather_conditions','Light_conditions', 'Road_surface_conditions','Road_surface_type','Casualty_class', 'Sex_of_casualty', 'Age_band_of_casualty', 'Casualty_severity','Pedestrian_movement','Cause_of_accident']

In [112]:
outputLabel.value_counts()
outputLabel = outputLabel.map({'Slight Injury' : 1, 'Serious Injury' : 2 ,'Fatal injury':3})

In [113]:
from sklearn.model_selection import train_test_split
X_train_ful, X_test_new, y_train_ful, y_test_new = train_test_split(imputed_features, outputLabel, test_size=0.15, random_state=8675309)

In [114]:
from sklearn.model_selection import train_test_split
X_train_new, X_val, y_train_new, y_val = train_test_split(X_train_ful, y_train_ful, test_size=0.15, random_state=8675309)

In [115]:
# imputed_features_new = imputed_features.drop(["Time",'Sex_of_driver','Vehicle_driver_relation','Area_accident_occured','Road_surface_conditions','Light_conditions','Lanes_or_Medians','Fitness_of_casuality','Types_of_Junction','Vehicle_movement','Pedestrian_movement','Casualty_class','Sex_of_casualty','Age_band_of_casualty'], axis=1)

In [116]:
# prepare input data
def prepare_inputs(X_train_new, X_val, X_test_new):
    for cat_col in cat_col_list:
      oe = LabelEncoder()
      print(cat_col)
      oe.fit(X_train_new[cat_col])
      X_train_new[cat_col] = oe.transform(X_train_new[cat_col])
      X_val[cat_col] = oe.transform(X_val[cat_col])
      X_test_new[cat_col] = oe.transform(X_test_new[cat_col])

In [117]:
prepare_inputs(X_train_new, X_val, X_test_new)

Type_of_vehicle
Day_of_week
Age_band_of_driver
Sex_of_driver
Educational_level
Vehicle_driver_relation
Driving_experience
Owner_of_vehicle
Service_year_of_vehicle
Defect_of_vehicle
Area_accident_occured
Lanes_or_Medians
Road_allignment
Types_of_Junction
Fitness_of_casuality
Work_of_casuality
Vehicle_movement
Type_of_collision
Weather_conditions
Light_conditions
Road_surface_conditions
Road_surface_type
Casualty_class
Sex_of_casualty
Age_band_of_casualty
Casualty_severity
Pedestrian_movement
Cause_of_accident


In [118]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [128]:
def run_exps(X_train: pd.DataFrame , y_train: pd.DataFrame, X_test: pd.DataFrame, y_test: pd.DataFrame, outputLabel) -> pd.DataFrame:
    '''
    Lightweight script to test many models and find winners
    :param X_train: training split
    :param y_train: training target vector
    :param X_test: test split
    :param y_test: test target vector
    :return: DataFrame of predictions
    '''    
    dfs = []
    models = [
              ('LogReg', LogisticRegression(solver='liblinear')), 
              ('RF', RandomForestClassifier()),
              ('KNN', KNeighborsClassifier()), 
              ('GNB', GaussianNB()),
              ('XGB', XGBClassifier())
            ]

    results = []
    names = []
    scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
    target_names = ['Slight Injury','Serious Injury','Fatal injury']
    for name, model in models:
        kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state=90210)
        cv_results = model_selection.cross_validate(model, X_train, y_train, cv=kfold, scoring=scoring)
        clf = model.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        y_probas = model.predict_proba(X_test)
        #   importances = model.feature_importances_

        print(name)
        print(classification_report(y_test, y_pred, target_names=target_names))
        run = wandb.init(project='my-scikit-integration', name=name)
        wandb.sklearn.plot_classifier(model, 
                              X_train, X_test, 
                              y_train, y_test, 
                              y_pred, y_probas, 
                              outputLabel, 
                              is_binary=True, 
                              model_name=name)

        wandb.finish()
#         print(confusion_matrix(y_test, y_pred, labels=target_names))

    # results.append(cv_results)
    # names.append(name)
    # this_df = pd.DataFrame(cv_results)
    # this_df['model'] = name
    # dfs.append(this_df)
    # final = pd.concat(dfs, ignore_index=True)
    # return final

In [120]:
X_train_new.drop(["Time"], inplace=True, axis=1)
X_val.drop(["Time"], inplace=True, axis=1)
X_test_new.drop(["Time"], inplace=True, axis=1)


In [138]:
X_train_new = X_train_new.astype(int)
X_val = X_val.astype(int)
X_test_new = X_test_new.astype(int)

In [139]:
run_exps(X_train_new, y_train_new, X_val, y_val, outputLabel)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Pr

LogReg
                precision    recall  f1-score   support

 Slight Injury       0.84      1.00      0.91      1312
Serious Injury       0.00      0.00      0.00       234
  Fatal injury       0.00      0.00      0.00        25

      accuracy                           0.84      1571
     macro avg       0.28      0.33      0.30      1571
  weighted avg       0.70      0.84      0.76      1571



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


wandb: 
wandb: Plotting LogReg.
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"
wandb: WARNING 2-dimensional feature importances array passed to plot_feature_importances. 2-dimensional and higher feature importances arrays are not currently supported. These importances will not be plotted.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Pr

...

wandb: 
wandb: Plotting GNB.
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  "X does not have valid feature names, but"
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  "X does not have valid feature names, but"
wandb: Logged summary metrics.
wandb: Logged class proportions.
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  "X does not have valid feature names, but"
wandb: WARNING This function only supports binary classification at the moment and therefore expe

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Pr

XGB
                precision    recall  f1-score   support

 Slight Injury       0.84      1.00      0.91      1312
Serious Injury       0.50      0.00      0.01       234
  Fatal injury       0.00      0.00      0.00        25

      accuracy                           0.84      1571
     macro avg       0.45      0.33      0.31      1571
  weighted avg       0.77      0.84      0.76      1571



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


wandb: 
wandb: Plotting XGB.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: WARNING This function only supports binary classification at the moment and therefore expects labels to be binary. Skipping calibration curve.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


0